## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. 현재 대화의 진행방식은 유저가 말하고, 에이전트가 말하는 방식입니다. 그렇기 때문에 유저가 먼저 말을 해야하는데, 처음 우리 앱을 들어온 유저는 우리가 어떤 것을 하는지 전혀 모르기 때문에 이에 대해 가이드를 주어야합니다.
2. 추천에 있어서는 에이전트가 유저를 분석하기 위해 여러 와인 선정 기준에 대해 질문을 합니다.(예시: 와인의 종류는 어떤것을 좋아하시나요?) 이런 질문에 대해서 후보 답변을 주는 것은 유저의 선택에 도움이 될 것 입니다.(예시: 1. 레드와인, 2. 화이트와인, 3. 스파클링)

#### 당신의 목표는 아래와 같습니다.
1. 유저가 실제로 우리앱을 쓰면서 대화 문맥 상에서 할만한 말들에 대한 후보를 생성합니다.

#### 생각할 수 있는 해결방식은 아래와 같습니다.
1. 적절한 유저 프롬프트를 작성하고, Langchain을 활용하여 유저의 예시 답변을 생성한다.
2. 미리 example들을 작성해두고, 이를 프롬프트에 추가하는 few shot방식을 통해 성능이 올라갈 수 있다.

#### 참고 사항
아래는 위의 작업을 수행하기 위한 좋은 자료들입니다. 위의 문제해결 전에 아래 자료들을 먼저 학습하는 것을 매우 강력하게 추천합니다.
- [프롬프트 엔지니어링 강의 2시간](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)
- [프롬프트 엔지니어링 가이드 documentation](https://www.promptingguide.ai/techniques)

-------------------------------------------------------------------------

### API 키 불러오기

In [1]:
import os
import configparser
import time
import copy

In [2]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [3]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
serper_api_key = config['SERPER']['SERPER_API_KEY']
serp_api_key = config['SERPAPI']['SERPAPI_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})
os.environ.update({'SERPER_API_KEY': serper_api_key})
os.environ.update({'SERPAPI_API_KEY': serp_api_key})

### Get User Prompt

In [4]:
from typing import List, Union
import re
import json

import pandas as pd
from langchain import SerpAPIWrapper, LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.document_loaders import DataFrameLoader, SeleniumURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.prompts import load_prompt, BaseChatPromptTemplate
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.vectorstores import DocArrayInMemorySearch, Chroma

유저 프롬프트 템플릿은 few_shot prompt를 사용하기 위해 [prefix template text file](./templates/user_prompt_prefix_template.txt)와 [suffix template text file](./templates/user_prompt_suffix_template.txt)로 나누어 작성합니다.

유저 프롬프트에 넣을 예시는 [examples text file](./templates/examples/user_prompt_examples.json)에 작성합니다.

그리고 [user prompt json file](./templates/user_prompt_template.json)을 통해 프롬프트를 불러옵니다.

In [45]:
# 파일로 부터 프롬프트 불러오기
user_response_prompt = load_prompt("./templates/user_response_prompt.json")

아래와 같이 format 메서드로 프롬프트를 확인할 수 있습니다.

In [46]:
example_conversation_history = """
User: 안녕하세요. <END_OF_TURN>
이우선: 무엇을 도와드릴까요? <END_OF_TURN>
User: 와인 추천해주세요. <END_OF_TURN>
이우선: 어떤 행사나 기념일을 위해 와인을 찾으시는지 알려주실 수 있으신가요? <END_OF_TURN>
"""

In [47]:
print(
    user_response_prompt.format(
        conversation_history=example_conversation_history,
    )
)

You are a user talking to a wine chatbot which can offer recommendations and Q&A.
Read the conversation history and generate candidate responses for the current user to say.
You can create as many candidate responses as you want.

Below is few examples of how to do this task.

conversation_history: 

user_response: 이번 주에 친구들과 모임이 있는데, 훌륭한 와인 한 병을 추천해줄래? | 입문자에게 좋은 와인을 추천해줄래? | 보르도와 부르고뉴 와인의 차이점은 뭐야?

conversation_history: 
User: 와인 하나만 추천해줘 <END_OF_TURN>
이우선: 고객님의 취향을 알아야 와인을 추천해드릴 수 있어요. 와인 종류 중 어떤 것을 선호하시나요? 레드, 화이트, 로제, 스파클링 중에서 선택해주세요. <END_OF_TURN>

user_response: 레드 와인 | 화이트 와인 | 로제 와인 | 스파클링

conversation_history: 
User: 와인 하나만 추천해주세요 <END_OF_TURN>
이우선: 고객님의 취향을 알아야 와인을 추천해드릴 수 있어요. 와인 종류 중 어떤 것을 선호하시나요? 레드, 화이트, 로제, 스파클링 중에서 선택해주세요. <END_OF_TURN>
User: 로제가 좋겠군 <END_OF_TURN>
이우선: 좋아요, 로제 와인을 좋아하시는군요! 로제 와인의 단맛 정도를 알려주시겠어요? 달달한 것을 선호하시나요, 아니면 적당한 단맛을 선호하시나요? <END_OF_TURN>

User response candidate: 달달한 와인 | 달지않은 와인 | 적당한 단맛의 와인 | 잘 모르겠어

Now generate a few candidate responses for 

In [48]:
# 랭체인 모델 선언, 랭체인은 언어모델과 프롬프트로 구성됩니다.
llm = ChatOpenAI(model='gpt-4', temperature=0.5)
user_response_chain = LLMChain(
    llm=llm,
    prompt=user_response_prompt, 
    verbose=True, # 과정을 출력할지
    )

Chain을 실행하기 위해서는 run 메서드를 실행한다.

In [49]:
user_responses = user_response_chain.run(
    {'conversation_history': example_conversation_history,}
)



> Entering new  chain...
Prompt after formatting:
You are a user talking to a wine chatbot which can offer recommendations and Q&A.
Read the conversation history and generate candidate responses for the current user to say.
You can create as many candidate responses as you want.

Below is few examples of how to do this task.

conversation_history: 

user_response: 이번 주에 친구들과 모임이 있는데, 훌륭한 와인 한 병을 추천해줄래? | 입문자에게 좋은 와인을 추천해줄래? | 보르도와 부르고뉴 와인의 차이점은 뭐야?

conversation_history: 
User: 와인 하나만 추천해줘 <END_OF_TURN>
이우선: 고객님의 취향을 알아야 와인을 추천해드릴 수 있어요. 와인 종류 중 어떤 것을 선호하시나요? 레드, 화이트, 로제, 스파클링 중에서 선택해주세요. <END_OF_TURN>

user_response: 레드 와인 | 화이트 와인 | 로제 와인 | 스파클링

conversation_history: 
User: 와인 하나만 추천해주세요 <END_OF_TURN>
이우선: 고객님의 취향을 알아야 와인을 추천해드릴 수 있어요. 와인 종류 중 어떤 것을 선호하시나요? 레드, 화이트, 로제, 스파클링 중에서 선택해주세요. <END_OF_TURN>
User: 로제가 좋겠군 <END_OF_TURN>
이우선: 좋아요, 로제 와인을 좋아하시는군요! 로제 와인의 단맛 정도를 알려주시겠어요? 달달한 것을 선호하시나요, 아니면 적당한 단맛을 선호하시나요? <END_OF_TURN>

User response candidate: 달달한 와인 | 달지않은 와인 | 적당한 단맛의 와인 |

In [50]:
user_responses

'생일 파티를 위한 와인 추천해주세요. | 결혼 기념일을 위한 와인 추천해주세요. | 친구와의 만남을 위한 와인 추천해주세요. | 저녁 식사를 위한 와인 추천해주세요.'